In [1]:
import pandas as pd
import numpy as np
import requests
import statsmodels.formula.api as smf
import time
from haversine import haversine, Unit

In [2]:
dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')
birm_disposal = pd.read_excel(r'../../data/raw/__Disposal( in period 09052025).xlsx')

C:\Users\User\AppData\Local\Temp\ipykernel_2400\2275096176.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')


In [3]:
category_map = {
    'classification_makes_grants_to_individuals': 'Grants_Related',
    'classification_makes_grants_to_organisations': 'Grants_Related',
    'classification_accommodation/housing': 'Housing_Infrastructure',
    'classification_provides_buildings/facilities/open_space': 'Housing_Infrastructure',
    'classification_education/training': 'Education_Research',
    'classification_sponsors_or_undertakes_research': 'Education_Research',
    'classification_children/young_people': 'Children_Youth',
    'classification_amateur_sport': 'Children_Youth',
    'classification_the_advancement_of_health_or_saving_of_lives': 'Health_Disability',
    'classification_disability': 'Health_Disability',
    'classification_people_with_disabilities': 'Health_Disability',
    'classification_human_rights/religious_or_racial_harmony/equality_or_diversity': 'Human_Rights_Advocacy',
    'classification_provides_advocacy/advice/information': 'Human_Rights_Advocacy',
    'classification_people_of_a_particular_ethnic_or_racial_origin': 'Human_Rights_Advocacy',
    'classification_religious_activities': 'Religion_Faith',
    'classification_acts_as_an_umbrella_or_resource_body': 'Religion_Faith',
    'classification_environment/conservation/heritage': 'Environment_Animals',
    'classification_animals': 'Environment_Animals',
    'classification_economic/community_development/employment': 'Economic_Social Development',
    'classification_general_charitable_purposes': 'Economic_Social_Development',
    'classification_provides_services': 'Economic_Social_Development',
    'classification_provides_human_resources': 'Economic_Social_Development',
    'classification_provides_other_finance': 'Economic_Social_Development',
    'classification_other_charitable_purposes': 'Economic_Social_Development',
    'classification_other_charities_or_voluntary_bodies': 'Economic_Social_Development',
    'classification_overseas_aid/famine_relief': 'Overseas_Famine_Relief',
    'classification_the_general_public/mankind': 'Other_Miscellaneous',
    'classification_other_defined_groups': 'Other_Miscellaneous',
    'classification_recreation': 'Other_Miscellaneous',
    'classification_elderly/old_people': 'Other_Miscellaneous',
    'classification_armed_forces/emergency_service_efficiency': 'Other_Miscellaneous',
    'classification_arts/culture/heritage/science': 'Other_Miscellaneous',
}
# Invert mapping: dummy_col -> group
dummy_to_group = {col: group for col, group in category_map.items()}

# Prepare a DataFrame to hold new dummy columns
group_dummies = pd.DataFrame(0, index=dataset.index, columns=sorted(set(dummy_to_group.values())))

# Assign each charity to the first matching group only (no double count)
for idx, row in dataset.iterrows():
    for col, group in dummy_to_group.items():
        if col in dataset.columns and row[col] == 1:
            group_dummies.at[idx, group] = 1
            break  # stop after first match
dataset = pd.concat([dataset, group_dummies], axis=1)


In [4]:
df = dataset[dataset['local_authority'] == 'Birmingham']

In [5]:
# Define geocoding function using postcodes.io
def geocode_postcode(postcode):
    postcode = postcode.replace(" ", "")
    url = f"https://api.postcodes.io/postcodes/{postcode}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 200 and data['result'] is not None:
            return data['result']['latitude'], data['result']['longitude']
    return None, None

# Unique postcodes from both datasets
charity_postcodes = df['postcode'].dropna().unique()
asset_postcodes = birm_disposal['POSTCODE'].dropna().unique()
all_postcodes = set(charity_postcodes).union(set(asset_postcodes))

# Geocode all unique postcodes
postcode_coords = {}
for pc in all_postcodes:
    lat, lon = geocode_postcode(pc)
    postcode_coords[pc] = {'latitude': lat, 'longitude': lon}

# Map back to original DataFrames
df['latitude'] = df['postcode'].map(lambda x: postcode_coords.get(x, {}).get('latitude'))
df['longitude'] = df['postcode'].map(lambda x: postcode_coords.get(x, {}).get('longitude'))

birm_disposal['latitude'] = birm_disposal['POSTCODE'].map(lambda x: postcode_coords.get(x, {}).get('latitude'))
birm_disposal['longitude'] = birm_disposal['POSTCODE'].map(lambda x: postcode_coords.get(x, {}).get('longitude'))


C:\Users\User\AppData\Local\Temp\ipykernel_2400\4170907305.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latitude'] = df['postcode'].map(lambda x: postcode_coords.get(x, {}).get('latitude'))
C:\Users\User\AppData\Local\Temp\ipykernel_2400\4170907305.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longitude'] = df['postcode'].map(lambda x: postcode_coords.get(x, {}).get('longitude'))


In [6]:
# Make sure you have asset coordinates as (lat, lon) tuples
asset_coords = birm_disposal[['latitude', 'longitude']].dropna().apply(tuple, axis=1).tolist()

# Function to get distance to nearest asset
def min_distance_to_asset(charity_lat, charity_lon):
    if pd.isna(charity_lat) or pd.isna(charity_lon):
        return None
    charity_point = (charity_lat, charity_lon)
    distances = [haversine(charity_point, asset_point, unit=Unit.KILOMETERS) for asset_point in asset_coords]
    return min(distances) if distances else None

# Apply to charity dataframe
df['distance_to_nearest_asset_km'] = df.apply(
    lambda row: min_distance_to_asset(row['latitude'], row['longitude']),
    axis=1
)


C:\Users\User\AppData\Local\Temp\ipykernel_2400\4030524278.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance_to_nearest_asset_km'] = df.apply(


In [19]:
def assign_treatment(distance):
    if pd.isna(distance):
        return 0
    return 1 if distance <= 1.0 else 0

df['treatment'] = df['distance_to_nearest_asset_km'].apply(assign_treatment)

C:\Users\User\AppData\Local\Temp\ipykernel_2400\3216631194.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['treatment'] = df['distance_to_nearest_asset_km'].apply(assign_treatment)


In [20]:
df['removed_post_disposal'] = ((df['removal_fy'] >= 2023)).astype(int)

C:\Users\User\AppData\Local\Temp\ipykernel_2400\1750751665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['removed_post_disposal'] = ((df['removal_fy'] >= 2023)).astype(int)


In [16]:
df

,registered_charity_number,charity_name,postcode,charity_status,charity_type,date_of_registration,date_of_removal,latest_income,charity_activities,charity_has_land,...,Housing_Infrastructure,Human_Rights_Advocacy,Other_Miscellaneous,Overseas_Famine_Relief,Religion_Faith,latitude,longitude,distance_to_nearest_asset_km,treatment,removed_post_disposal
353,1000355,JAMI MOSQUE AND ISLAMIC CENTRE (BIRMINGHAM) TR...,B10 0LL,active,Charitable company,1990-11-20,NaN,1904091.0,Our charity also runs: 1. Muslim marriage regi...,True,...,1,0,0,0,0,52.468427,-1.855619,0.548038,1,0
463,1000465,THE ARAB WELFARE AND COMMUNITY CENTRE,B12 9SU,inactive,Other,1990-09-28,2023-10-19,0.0,We are currently supplying a range of support ...,NaN,...,0,0,0,0,0,52.460715,-1.891320,0.206950,0,1
494,1000496,S B AND H M JACKSON CHARITABLE TRUST,B2 5SN,inactive,Trust,1990-10-05,2025-02-26,0.0,General charitable purposes.,False,...,0,0,0,0,0,52.480501,-1.900371,0.809559,1,1
506,1000508,WITTON AND ASTON CHURCHES YOUTHWORK AGENCY,B32 3HU,inactive,Other,1990-10-05,2015-12-01,0.0,WACYA seeks to support member and other organi...,NaN,...,0,0,0,0,0,52.439207,-1.994228,0.295139,0,0
596,1000598,SUPPORT AFTER MURDER AND MANSLAUGHTER,B5 7RN,active,Other,1990-10-18,NaN,214000.0,SAMM is an independent voluntary organisation....,False,...,0,0,0,0,0,52.454094,-1.907155,0.732228,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347697,800234,THE ORAL AND DENTAL RESEARCH TRUST,B5 7EG,active,Other,1988-11-22,NaN,52150.0,Support Oral and Dental Research activity thro...,False,...,0,0,0,0,0,52.449991,-1.914476,1.340718,0,0
348974,801552,ONE DANCE UK,B5 4TB,active,Charitable company,1994-11-10,NaN,859981.0,"ONE DANCE UK IS THE NATIONAL VOICE FOR DANCE, ...",False,...,0,0,0,0,0,52.474297,-1.898618,0.346228,0,0
349763,802394,THE BETTY KENSIK FUND,B17 0PN,active,Trust,1990-07-02,NaN,20985.0,Support Animal Welfare,False,...,0,0,0,0,0,52.448942,-1.950895,0.502953,1,0
350340,803009,GOWLING WLG (UK) CHARITABLE TRUST,B4 6WR,active,Other,1990-03-09,NaN,166063.0,Income is to be applied for such charitable in...,False,...,0,0,0,0,0,52.484141,-1.898832,0.493795,0,0


In [21]:
model = smf.logit(
    formula="""
        removed_post_disposal ~ treatment 
        + registration_fy 
        + Housing_Infrastructure
        + charity_has_land
    """,
    data=df
).fit()

model.summary()


Optimization terminated successfully.
         Current function value: 0.154277
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     removed_post_disposal   No. Observations:                 2674
Model:                             Logit   Df Residuals:                     2669
Method:                              MLE   Df Model:                            4
Date:                   Wed, 02 Jul 2025   Pseudo R-squ.:                 0.01825
Time:                           17:57:34   Log-Likelihood:                -412.54
converged:                          True   LL-Null:                       -420.21
Covariance Type:               nonrobust   LLR p-value:                  0.004047
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   29.5381     11.581      2.551      0.011       6.840      52.236
charity_has_land[T.True]    -0.5780      0.267     -2.169      0.030      -1.100      -0.056
treatment                    0.4490      0.430      1.044      0.296      -0.394       1.292
registration_fy             -0.0165      0.006     -2.844      0.004      -0.028      -0.005
Housing_Infrastructure      -0.2950      0.281     -1.051      0.293      -0.845       0.255
============================================================================================
"""